In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nilearn
import h5py
import nibabel
from nilearn.regions import Parcellations
import os

base_path = '/kaggle/input/trends-assessment-prediction/'

def kmeans_clustering(mat_path, mask_path, method='kmeans', n_parcels=50, smoothing_fwhm=10. ):
    
    # import nilearn h5py numpy nibabel
    # read in mat file
    mat = h5py.File(mat_path, 'r')
    data = mat['SM_feature']
    
    # convert to np array and flip axis
    data = data[()]
    array = np.moveaxis(data, [0,1,2,3], [3,2,1,0])
    print(array.shape)
    
    # convert to nifti load mask
    mask_nimg = nilearn.image.load_img(mask_path)
    nifti = nibabel.nifti1.Nifti1Image(array, affine=mask_nimg.affine)
    
    # call parcellations
    clusterer = Parcellations(method=method, n_parcels=n_parcels, mask=mask_nimg,
                       standardize=True, smoothing_fwhm=smoothing_fwhm,
                       memory_level=1,
                       verbose=1)
    # Call fit on functional dataset: single subject (less samples)
    clusterer.fit(nifti)
    # save to file
    file_name = mat_path.split('/')
    file_name = file_name[-1].split('.')
    clusterer.labels_img_.to_filename(file_name[0] + '.nii')
    print("saving " + file_name[0])